<a href="https://colab.research.google.com/github/vyom10445/toxicity-classifier/blob/main/notebooks/Toxicity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow pandas matplotlib scikit-learn gradio

In [ ]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np